In [1]:
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
import re
import pandas as pd

In [2]:
headers = {
    'User-Agent': 'Shoe Scraper/1.0 (+https://a10g.com/shoe-scraper)'
}

## News
Get news articles.
Assign article/title to variables.

In [3]:
#newsurl = "https://mars.nasa.gov/news/"
news_url="https://mars.nasa.gov/news?page=0&amp;per_page=40&amp;order=publish_date+desc%2Ccreated_at+desc&amp;search=&amp;category=19%2C165%2C184%2C204&amp;blank_scope=Latest"
news_response = requests.get(news_url, headers=headers)
soup = bs(news_response.text, 'lxml')

In [4]:
#print(soup.prettify())

In [5]:
# Find article titles
article_titles = soup.find_all('div', class_='content_title')
#article_titles
news_titles = []
for article in article_titles:
    title = article.find('a')
    title_text = title.text.strip()
    #print(title_text)
    news_titles.append(title_text)
news_titles
len(news_titles)

6

In [6]:
# Find paragraph text
paragraphs = soup.find_all('div', class_='rollover_description')
#paragraphs
news_content = []
for paragraph in paragraphs:
    ptext = paragraph.find('div')
    newsp = ptext.text.strip()
    #print(newsp)
    news_content.append(newsp)
news_content
len(news_content)

6

In [7]:
all_latest_news = dict(zip(news_titles, news_content))
latest_title = news_titles[0]
latest_content = news_content[0]

## JPL Featured Image

In [8]:
#path = !which chromedriver
#print(path[0])
#path_to_chrome = path[0]
#mars_images_browser = Browser(path_to_chrome, headless=False)
mars_images_browser = Browser('chrome', headless=False)
jpl_url = 'https://www.jpl.nasa.gov'
space_img = 'spaceimages'
qstring = '?search=&category=Mars'
nasa_url = jpl_url + "/" + space_img + "/" + qstring

In [9]:
#print(jpl_url + "/" + space_img + "/" + qstring) 
nasa_url

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [10]:
mars_images_browser.visit(nasa_url)

In [11]:
mars_images_html = mars_images_browser.html
jpl_soup = bs(mars_images_html, 'html.parser')

In [12]:
#print(jpl_soup.body.prettify())

In [13]:
# Trying to get to this:
# https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18847_hires.jpg

featured_image = jpl_soup.find_all('a', class_='button fancybox')
featured_image[0].attrs
#jpl_img = featured_image[0]['data-link']
jpl_img = featured_image[0]['data-fancybox-href']
jpl_img_url = jpl_url + jpl_img 
jpl_img_url

jpl_img_url_step1 = re.sub(r'mediumsize', 
                        r'largesize',
                        jpl_img_url, re.IGNORECASE)
jpl_img_url_lg = re.sub(r'_ip', 
                        r'_hires',
                        jpl_img_url_step1, re.IGNORECASE)
jpl_img_url_lg
mars_images_browser.quit()

## Mars Weather Twitter

In [14]:
twiturl="https://twitter.com/marswxreport?lang=en"
twitresponse = requests.get(twiturl)
twitsoup = bs(twitresponse.text, 'lxml')

In [15]:
#print(soup.prettify())

In [16]:
# Find weather tweets with BeautifulSoup
tweets = twitsoup.find_all('p', class_='TweetTextSize')
weather_on_mars = tweets[0].text
weather_on_mars

'InSight sol 99 (2019-03-07) low -95.8ºC (-140.4ºF) high -13.3ºC (8.1ºF)\nwinds from the SW at 4.2 m/s (9.5 mph) gusting to 12.0 m/s (26.8 mph)\npressure at 7.20 hPapic.twitter.com/XRi0faFSv5'

## Space Facts

In [17]:
space_facts = 'https://space-facts.com/mars/'
sf_table = pd.read_html(space_facts)

In [18]:
sf_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [19]:
mars_table = pd.DataFrame(sf_table[0])
mars_table.columns=['Attribute', 'Value']
#mars_facts = sf_table[0].to_html()
#print(mars_facts)
mars_table
mars_facts = mars_table.to_html()
#print(mars_facts)

## Hemispheres

In [20]:
# Manual fallback
#ch = 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'
#vmh = 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'
#sh = 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'
#smh = 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'
#
#hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": vmh},
#    {"title": "Cerberus Hemisphere", "img_url": ch},
#    {"title": "Schiaparelli Hemisphere", "img_url": sh},
#    {"title": "Syrtis Major Hemisphere", "img_url": smh},
#]

In [21]:
usgs_browser = Browser('chrome', headless=False)
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
usgs_browser.visit(usgs_url)

In [22]:
mh_html = usgs_browser.html
mh_soup = bs(mh_html, 'html.parser')

In [23]:
#print(mh_soup.prettify())

In [24]:
mh = mh_soup.find_all('div', class_='description')
mh

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [25]:
mh_image_urls = []
for image in mh:
    h_url = image.find('a', class_='itemLink')
    h = h_url.get('href')
    h_link = 'https://astrogeology.usgs.gov' + h
    print(h_link)

    usgs_browser.visit(h_link)
    h_image_dict = {}

    h_html = usgs_browser.html
    h_soup = bs(h_html, 'html.parser')

    h_link = h_soup.find('a', text='Original').get('href')

    h_title = h_soup.find('h2', class_='title').text.replace(' Enhanced', '')

    h_image_dict['title'] = h_title
    h_image_dict['img_url'] = h_link

    mh_image_urls.append(h_image_dict)

print(mh_image_urls)
usgs_browser.quit()

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]


In [28]:
# This is for checking the output when using the "restart and run all" option

#print("Latest News Headline: " + str(latest_title))
#print("Latest News Body: " + str(latest_content))

#print("")
#print("Weather on Mars: " + str(weather_on_mars))
#print("HiRes JPL URL: " + str(jpl_img_url_lg))

#print("")
#print("Mars Facts: " + mars_facts)

#print("")

#print("Mars Hemisphers: ")
#print(mh_image_urls)

Latest News Body: NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.

Weather on Mars: InSight sol 99 (2019-03-07) low -95.8ºC (-140.4ºF) high -13.3ºC (8.1ºF)
winds from the SW at 4.2 m/s (9.5 mph) gusting to 12.0 m/s (26.8 mph)
pressure at 7.20 hPapic.twitter.com/XRi0faFSv5
HiRes JPL URL: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19347_hires.jpg

Mars Facts: <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Fact</th>
      <th>More Fact!</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <t

In [30]:
mars_bars = {
    'latest_news':{
        'headline':latest_title,
        'content':latest_content
    },
    'current_weather':weather_on_mars,
    'jpl_featured_image':jpl_img_url_lg,
    'facts':mars_facts,
    'hemispheres':mh_image_urls    
}

#print("")
#print("Weather on Mars: " + str(weather_on_mars))
#print("HiRes JPL URL: " + str(jpl_img_url_lg))

#print("")
#print("Mars Facts: " + mars_facts)

#print("")

#print("Mars Hemisphers: ")
#print(mh_image_urls)
print(mars_bars)

{'latest_news': {'headline': "NASA's Opportunity Rover Mission on Mars Comes to End", 'content': "NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet."}, 'current_weather': 'InSight sol 99 (2019-03-07) low -95.8ºC (-140.4ºF) high -13.3ºC (8.1ºF)\nwinds from the SW at 4.2 m/s (9.5 mph) gusting to 12.0 m/s (26.8 mph)\npressure at 7.20 hPapic.twitter.com/XRi0faFSv5', 'jpl_featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19347_hires.jpg', 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact</th>\n      <th>More Fact!</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2

In [31]:
# one time output
! jupyter nbconvert --to script --template basic mission_to_mars.ipynb --output scrape_mars

[NbConvertApp] Converting notebook mission_to_mars.ipynb to script
[NbConvertApp] Writing 6243 bytes to scrape_mars.py
